In [1]:
import findspark
findspark.init()
#from Ipython.core.interactiveShell import InteractiveShell
#InteractiveShell.ast_node_interactivity = 'all'

import pyspark
from pyspark import SparkContext, SparkConf
from pyspark.sql import SparkSession, SQLContext, Row

from pyspark.ml.feature import HashingTF, IDF, Tokenizer, StopWordsRemover, CountVectorizer
from pyspark.ml.tuning import CrossValidator, ParamGridBuilder
from pyspark.sql.functions import col, when, explode
import pyspark.sql.functions as F

from pyspark.ml.linalg import Vectors

import pyodbc
#from Ipython.display import display, FileLink, FileLinks

import os
import pandas as pd
from pandas import DataFrame
#pd.option.display.max_columns = None
import numpy as np
import lightgbm as lgb
import xgboost as xgb

In [ ]:
spark =SparkSession\
   .builder\
   .appName("test")\
   .enableHiveSupport().getOrCreate()

sc= spark.sparkContext
sqlContext= SQLContext(sc)

findspark.find()

In [ ]:
df = sqlContext.read.format("csv")\
   .option("header", "true")\
   .load("C:/Users/affiqazrin/Desktop/dataset/Data_FinalProject.csv")

In [ ]:
# Convert Feature Types
df.createOrReplaceTempView("df")

df2 = spark.sql("select \
                    cast(age as int) as age, \
                    cast(job as string) as job, \
                    cast(marital as string) as marital, \
                    cast(education as string) as education, \
                    cast(default as string) as default, \
                    cast(housing as string) as housing, \
                    cast(loan as string) as loan, \
                    cast(contact as string) as contact, \
                    cast(day_of_week as string) as day, \
                    cast(month as string) as month, \
                    cast(duration as int) as duration, \
                    cast(campaign as int) as campaign, \
                    cast(pdays as int) as pdays, \
                    cast(previous as int) as previous, \
                    cast(poutcome as string) as poutcome, \
                    cast(y as string) as deposit \
                from df")

# Data Types
df2.dtypes
[('age', 'int'),
 ('job', 'string'),
 ('marital', 'string'),
 ('education', 'string'),
 ('default', 'string'),
 ('housing', 'string'),
 ('loan', 'string'),
 ('contact', 'string'),
 ('day', 'string'),
 ('month', 'string'),
 ('duration', 'int'),
 ('campaign', 'int'),
 ('pdays', 'int'),
 ('previous', 'int'),
 ('poutcome', 'string'),
 ('deposit', 'string')]

In [ ]:
df2.show()

In [ ]:
#target (y=deposit)
df.groupBy("y").count().show()

In [ ]:
#categorical attrib(11)
categorical_cols = ["job",
                    "marital",
                    "education",
                    "default",
                    "housing",
                    "loan",
                    "contact",
                    "day",
                    "month",
                    "poutcome",
                    "deposit"]
 
#numerical attrib(5)
numeric_cols = ["age",
                "duration",
                "campaign",
                "pdays",
                "previous"]

In [ ]:
from pyspark.ml import Pipeline
from pyspark.ml.feature import StringIndexer, OneHotEncoder, VectorAssembler
stages = []

#iterate through all categorical values
for cols in categorical_cols:
    #create a string indexer for those categorical values and assign a new name including the word 'Index'
    stringIndexer = StringIndexer(inputCol = cols,
                                  outputCol = cols + '_index')

    oneHotEncoder = OneHotEncoder(inputCol = cols + '_index',
                                  outputCol = cols + '_classVec').setDropLast(False)

    #append the string Indexer to our list of stages
    stages += [stringIndexer, oneHotEncoder]
    
assembler_inputs = [cols + "_classVec" for cols in categorical_cols] + numeric_cols

assembler = VectorAssembler(inputCols=assembler_inputs, outputCol="features")
stages += [assembler]
    
#Create the pipeline. Assign the satges list to the pipeline key word stages
pipeline = Pipeline(stages = stages)

#fit the pipeline to our dataframe
pipelineModel = pipeline.fit(df2)

#transform the dataframe
df3= pipelineModel.transform(df2)

In [ ]:
df3.show()

In [ ]:
df3.toPandas().to_csv('Data_FinalProject_READY.csv')